In [5]:
# -*- coding: utf-8 -*-
import matplotlib.pyplot as plt
import numpy as np
import h5py

# Konfiguracja wykresów
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams.update({'font.size': 12})

# -----------------------------------------------------------------------------
# Krok 1: Wczytanie danych z pliku .mat (HDF5)
# -----------------------------------------------------------------------------
try:
    with h5py.File('allFaces.mat', 'r') as mat_contents:
        # Pobranie danych z odpowiednią transpozycją
        faces = np.array(mat_contents['faces']).T  # Wymiary (liczba pikseli, liczba zdjęć)
        m = int(mat_contents['m'][0])              # Wysokość obrazu
        n = int(mat_contents['n'][0])              # Szerokość obrazu
        nfaces = np.array(mat_contents['nfaces']).flatten()  # Liczba twarzy na osobę
        
except Exception as e:
    print(f"Błąd wczytywania pliku: {e}")
    raise

# -----------------------------------------------------------------------------
# Krok 2: Przygotowanie danych treningowych
# -----------------------------------------------------------------------------
# Używamy pierwszych 36 osób do treningu
trainingFaces = faces[:, :np.sum(nfaces[:36])]

# Obliczanie średniej twarzy
avgFace = np.mean(trainingFaces, axis=1)

# Odejmowanie średniej
X = trainingFaces - np.tile(avgFace, (trainingFaces.shape[1], 1)).T

# -----------------------------------------------------------------------------
# Krok 3: Analiza PCA (Eigenfaces)
# -----------------------------------------------------------------------------
# Dekompozycja SVD
U, S, VT = np.linalg.svd(X, full_matrices=False)

# Obliczanie skumulowanej wariancji
total_variance = np.sum(S**2)
explained_variance = (S**2) / total_variance
cumulative_variance = np.cumsum(explained_variance)

# Znajdź minimalne r dla >16% wariancji
required_variance = 0.16
r = np.argmax(cumulative_variance > required_variance) + 1
print(f"Minimalna liczba eigenfaces (r): {r}")

# -----------------------------------------------------------------------------
# Krok 4: Wizualizacja
# -----------------------------------------------------------------------------
# Wykresy wartości osobliwych
plt.figure()
plt.semilogy(S)
plt.title("Wartości osobliwe (log scale)")
plt.xlabel("Numer komponentu")
plt.ylabel("Wartość")
plt.show()

plt.figure()
plt.plot(cumulative_variance)
plt.axhline(required_variance, color='r', linestyle='--')
plt.axvline(r, color='g', linestyle='--')
plt.title("Skumulowana wariancja")
plt.xlabel("Liczba komponentów")
plt.ylabel("Wyjaśniona wariancja")
plt.show()

# -----------------------------------------------------------------------------
# Krok 5: Rekonstrukcja testowej twarzy
# -----------------------------------------------------------------------------
# Pierwsza twarz osoby 37 (spoza zbioru treningowego)
testFace = faces[:, np.sum(nfaces[:36])]
testFaceMS = testFace - avgFace

# Rekonstrukcja z r eigenfaces
reconFace = avgFace + U[:, :r] @ (U[:, :r].T @ testFaceMS)

# Porównanie oryginału i rekonstrukcji
fig, ax = plt.subplots(1, 2)
ax[0].imshow(testFace.reshape(m, n).T, cmap='gray')
ax[0].set_title('Oryginalna twarz (37. osoba)')
ax[0].axis('off')

ax[1].imshow(reconFace.reshape(m, n).T, cmap='gray')
ax[1].set_title(f'Rekonstrukcja (r={r})')
ax[1].axis('off')
plt.tight_layout()
plt.show()

# -----------------------------------------------------------------------------
# Krok 6: Projekcja osób 2 i 7 na PC5 i PC6 (dodatkowa analiza)
# -----------------------------------------------------------------------------
P1num = 2  # Osoba 2
P2num = 7  # Osoba 7

# Pobranie danych
P1 = faces[:, np.sum(nfaces[:P1num-1]):np.sum(nfaces[:P1num])]
P2 = faces[:, np.sum(nfaces[:P2num-1]):np.sum(nfaces[:P2num])]

# Normalizacja
P1 = P1 - np.tile(avgFace, (P1.shape[1], 1)).T
P2 = P2 - np.tile(avgFace, (P2.shape[1], 1)).T

# Projekcja na PC5 i PC6 (indeksy 4 i 5 w Pythonie)
PCAmodes = [4, 5]
PCACoordsP1 = U[:, PCAmodes].T @ P1
PCACoordsP2 = U[:, PCAmodes].T @ P2

# Wykres
plt.figure()
plt.scatter(PCACoordsP1[0,:], PCACoordsP1[1,:], marker='d', color='k', label='Osoba 2')
plt.scatter(PCACoordsP2[0,:], PCACoordsP2[1,:], marker='^', color='r', label='Osoba 7')
plt.xlabel("PC5")
plt.ylabel("PC6")
plt.legend()
plt.title("Przestrzeń PCA")
plt.show()

Błąd wczytywania pliku: Unable to open file (file signature not found)


OSError: Unable to open file (file signature not found)